# Logistic Regression

In [1]:
import pandas as pd
import numpy as np

## Taking the AAPL for example to investigate the effect

#### Monthly

In [43]:
freq = 'Daily'
stock = 'AAPL'
price = pd.read_csv('../encode_price/'+freq+'/'+stock+'.csv')
y = price.direction.values
predictors = pd.read_csv('../predictors/Merged/'+freq+'/'+stock+'.csv',index_col='Date')
predictors.fillna(0,inplace=True)
X = predictors.values

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [45]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)
pipe = make_pipeline(StandardScaler(),LogisticRegression())
pipe.fit(X_train,y_train)
pipe.score(X_test,y_test)

0.5984251968503937